In [2]:
from model import load_untrained_bert_base, get_bert_config
from transformers import AutoConfig, BertForMaskedLM, AutoModelForSequenceClassification

/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def print_config_diff(config1,config2):
    c1 = config1.to_dict()
    c2 = config2.to_dict()
    for key in (set(c1.keys())|set(c2.keys())):
        v1 = None
        if key in c1:
            v1 = c1[key]
        v2 = None
        if key in c2:
            v2 = c2[key]
        if str(v1)!=str(v2):
            print(key, v1, v2)

In [4]:
print_config_diff(get_bert_config("small"),get_bert_config("small12h"))

hidden_size 512 504
num_attention_heads 8 12


In [5]:
from transformers.models.bert.modeling_bert import BertEmbeddings, BertEncoder,BertLayer, BertAttention, BertIntermediate, BertOutput

In [8]:
from modeling.bert import prepare_bert_for_quantization


ternary_bert = AutoModelForSequenceClassification.from_config(get_bert_config("base"))
prepare_bert_for_quantization(ternary_bert)

models = {
    "BERT\\textsubscript{BASE}": AutoModelForSequenceClassification.from_config(get_bert_config("base")),
    "BERT\\textsubscript{KD4}": AutoModelForSequenceClassification.from_config(get_bert_config("TinyBERT")),
    "BERT\\textsubscript{SMALL}": AutoModelForSequenceClassification.from_config(get_bert_config("small")),
    "BERT\\textsubscript{MINI}": AutoModelForSequenceClassification.from_config(get_bert_config("mini")),
    "TernaryBERT": ternary_bert,
}

In [23]:
import torch
from modeling.quantization import QuantizedEmbedding, QuantizedLinear


def calculate_memory(module):
    if isinstance(module,QuantizedEmbedding):
        return module.weight.numel()*2/8+module.weight.shape[0]*32
    elif isinstance(module,QuantizedLinear):
        return module.weight.numel()*2/8+4+module.bias.numel()*4
    else:
        size = sum([p.element_size()*p.numel() for p in module.parameters(recurse=False)])
        for child_module in module.children():
            size += calculate_memory(child_module)
    #size_in_bytes = sum([p.element_size()*p.numel() for p in module.parameters()])
    return size

30291948.0

In [33]:
from contextlib import redirect_stderr
from fvcore.nn import FlopCountAnalysis
import io
f = io.StringIO()
with redirect_stderr(f):
    for name,model in models.items():
        flops = FlopCountAnalysis(model, 100*torch.ones((32,128)).long())
        print(f"{name}: {calculate_memory(model)/1e6:.1f} MB, {flops.total()/1e9:.1f} GFLOPs")

BERT\textsubscript{BASE}: 437.9 MB, 358.0 GFLOPs
BERT\textsubscript{KD4}: 57.4 MB, 20.0 GFLOPs
BERT\textsubscript{SMALL}: 115.1 MB, 53.8 GFLOPs
BERT\textsubscript{MINI}: 44.7 MB, 14.0 GFLOPs


RuntimeError: "bitwise_or_cpu" not implemented for 'Float'

In [ ]:
def print_size(size):
    return str(size//1024 /1024)+" MB"

def print_numel(size):
    return str(size//1000 /1000)+" M"
    

In [19]:
print_numel(get_numel(BertForMaskedLM(get_bert_config("TinyBERT"))))

'14.381 M'

In [25]:
print("Embeddings size:", print_size(get_size(model.bert.embeddings)))
print("Encoder size:", print_size(get_size(model.bert.encoder)))

Embeddings size: 59.673828125 MB
Encoder size: 47.103515625 MB


In [8]:
get_size(model.bert.encoder.layer[0])

28351488

In [9]:
model.bert.encoder

BertEncoder(
  (layer): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=3072, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [10]:
from pthflops import count_ops

In [11]:
dataset["train"].select([0])

NameError: name 'dataset' is not defined

In [ ]:
count_ops(model,dataset["train"].select([0]))

AttributeError: 'dict' object has no attribute 'device'

In [ ]:
next(iter(dataset["train"].select([0])))

{'labels': tensor(0),
 'input_ids': tensor([  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  